# Pearson 

In [1]:
import pandas as pd
import numpy as np
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve 
from sklearn.linear_model import LogisticRegression
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
blacklist = ['NONE','age']

In [3]:
df = pd.read_csv("Covid-19_dataset.csv",header=1)


df['is_Male'] = pd.get_dummies(df['Gender'])['MALE']#one hot encoding gender so male = #1 and female = 0

df = pd.concat([df, pd.get_dummies(df['Blood_Group'])], axis=1)
# df = pd.concat([df, pd.get_dummies(df['CONTROL_blood_group'])], axis=1) # needed to remove may not be needed
# df = pd.concat([df, pd.get_dummies(df['Risk_Factor'])], axis=1)
df = pd.concat([df, pd.get_dummies(df['Infection_type'])], axis=1)

blood_types = pd.get_dummies(df['Blood_Group']).columns


df = df.drop(['CONTROL_blood_group','Risk_Factor','Infection_type','Gender','Place_of_isolation','Patient_In','Patient_out',' Incubation period (days)'],axis=1)
columns = df.columns



# getting rid of blank values
print("before",len(df))
for name in columns:
    df = df[df[name] != " "]
    df = df.dropna()
    
label_Severeity = df['Severeity'] #Severeity vs Outcome
label_Outcome = df[["Outcome"]]





df = df.drop(['Severeity','Outcome'],axis=1)


blood = pd.DataFrame(df["Blood_Group"])
df = df.drop(['Blood_Group'],axis=1)


df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

df = df.join(blood)
columns = df.columns


print("after",len(df))

df

before 5668
after 5641


,Age,Fever,Chills,Cough,Dyspnea,Anosmia_Ageusia,Loss_of_appetite,Asthenia,Headache,Muscle_ache,...,AB-,B+,B-,O+,O-,Asymptomatic,Asymtomatic,Symptomatic,Symtomatic,Blood_Group
0,61.0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,A+
1,49.0,1,0,1,0,1,1,1,1,1,...,0,0,0,1,0,0,0,1,0,O+
2,88.0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,A+
3,56.0,1,0,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,A+
4,41.0,0,0,1,0,1,1,1,1,1,...,0,1,0,0,0,0,0,1,0,B+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,65.0,1,0,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,1,0,B+
5664,32.0,1,1,1,1,1,1,1,1,1,...,0,1,0,0,0,0,0,1,0,B+
5665,28.0,1,1,1,1,1,1,0,1,1,...,0,1,0,0,0,0,0,1,0,B+
5666,40.0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,A+


In [4]:
all_features = df.columns

In [5]:
map_mortality = {"Died":0,"Recovered":1}
label_Outcome_mapped = deepcopy(label_Outcome)
label_Outcome_mapped['Outcome']=label_Outcome_mapped['Outcome'].map(map_mortality)

In [6]:
TierOneFeature = ['Age', 'Cyanosis', 'AT', 'COPD', 'DM', 'DMHT', 'HT', 'NONE']
TierTwoFeature = ['Sore_throat', 'Diarrhea', 'AP']
TierThreeFeature = ['CA', 'Headache', 'ARF', 'CKD', 'COPDDM', 'Cough']

coreFeatures = TierOneFeature+TierTwoFeature+TierThreeFeature+['Blood_Group']

In [7]:
#temp
coreFeatures = all_features
coreFeatures = set(coreFeatures) - set(['AT', 'COPD', 'DM', 'DMHT', 'HT', 'NONE', 'AP', 'CA', 'ARF', 'CKD', 'COPDDM'])
coreFeatures = list(coreFeatures)

In [8]:
df=df[coreFeatures]

In [9]:
coreFeatures

['Symtomatic',
 'Fever ',
 'O-',
 'Cyanosis',
 'Symptomatic',
 'Dyspnea',
 'Asymtomatic',
 'Age',
 'AB+',
 'Blood_Group',
 'Diarrhea',
 'O+',
 'Chills',
 'Asthenia',
 'B-',
 'Rhinorrhea',
 'Sore_throat',
 'A+',
 'Nausea_vomiting',
 'is_Male',
 'Headache',
 'AB-',
 'Cough',
 'A-',
 'Loss_of_appetite',
 'Anosmia_Ageusia',
 'B+',
 'Asymptomatic',
 'Muscle_ache']

In [10]:
# df=df.drop(blacklist,axis=1)

# 

In [11]:
def rank(predictor,blood_type):
    
    
    
    df_temp = df[df['Blood_Group'] == blood_type]
    X = df_temp.drop(['Blood_Group'],axis=1)

    X = pd.DataFrame(X[predictor])
    X['Outcome'] = label_Outcome_mapped
    
    corr_val = X['Outcome'].corr(X[predictor])
    
    return corr_val





blood_types = ["A+","A-","AB+","B+","B-","O+","O-"]

dataframes = []

BloodType_df = pd.DataFrame()

for blood_type in blood_types:
    print(blood_type,"-----------------------")
    list_of_AUC = []
    for feature in df.columns:
        if feature == "Blood_Group":
            continue
        #["Age"," Incubation period (days)","Cyanosis","AP","ARF","AT","COPD","DM","DMHT","HT","NONE"]:
        list_of_AUC.append({blood_type+" Feature":feature, blood_type+" Corr" : rank(feature,blood_type)})

        
    df_temp = pd.DataFrame(list_of_AUC)
    df_temp = df_temp.sort_values(by=[blood_type+' Corr',blood_type+' Feature'], ascending=False)
    
#     df = df.dropna(axis=0, how='any')
    
    df_temp = df_temp.reset_index()
    
    
    
    dataframes.append(df_temp)
    
    
    BloodType_df[blood_type+' Feature'] = df_temp[blood_type+' Feature']
    BloodType_df[blood_type+' Corr'] = df_temp[blood_type+' Corr']
    
    #print(df_temp)



#https://towardsdatascience.com/feature-selection-with-boruta-in-python-676e3877e596



A+ -----------------------
A- -----------------------
AB+ -----------------------
B+ -----------------------
B- -----------------------
O+ -----------------------
O- -----------------------


In [12]:
BloodType_df.head(10)

,A+ Feature,A+ Corr,A- Feature,A- Corr,AB+ Feature,AB+ Corr,B+ Feature,B+ Corr,B- Feature,B- Corr,O+ Feature,O+ Corr,O- Feature,O- Corr
0,Diarrhea,0.036801,Headache,0.394591,Rhinorrhea,0.057374,Cough,0.068726,Diarrhea,0.131764,is_Male,0.060103,Headache,0.305556
1,Anosmia_Ageusia,0.015020,Cough,0.191854,Cough,0.030780,is_Male,0.038256,Anosmia_Ageusia,0.115584,Chills,0.045931,Chills,0.148690
2,Dyspnea,0.010545,Diarrhea,0.114198,Fever,0.028949,Muscle_ache,0.021706,Headache,0.065705,Anosmia_Ageusia,0.043874,Sore_throat,0.096511
3,Asymtomatic,0.010479,Muscle_ache,0.059803,Diarrhea,0.026463,Chills,0.021175,Chills,0.052152,Diarrhea,0.034310,Nausea_vomiting,0.033162
4,Asymptomatic,0.006621,Rhinorrhea,0.059803,Asymptomatic,0.019718,Asymptomatic,0.012973,Rhinorrhea,0.030995,Rhinorrhea,0.015196,Asthenia,-0.023729
5,Nausea_vomiting,0.004858,Nausea_vomiting,0.055945,Loss_of_appetite,0.019406,Asthenia,0.010293,is_Male,0.028433,Cough,0.010249,Dyspnea,-0.033787
6,Fever,0.003743,is_Male,0.021997,Asymtomatic,0.017060,Asymtomatic,0.010037,Asymptomatic,0.011865,Asymtomatic,0.007862,Fever,-0.054554
7,Cough,-0.001407,Chills,-0.013585,Chills,0.016623,Diarrhea,-0.000328,Muscle_ache,0.000968,Asymptomatic,0.006417,Cough,-0.054554
8,Loss_of_appetite,-0.001882,Dyspnea,-0.033346,Headache,0.000923,Rhinorrhea,-0.001914,Symptomatic,-0.011865,Symtomatic,0.004536,Anosmia_Ageusia,-0.060193
9,is_Male,-0.005349,Anosmia_Ageusia,-0.033346,Muscle_ache,-0.014804,Loss_of_appetite,-0.005282,Loss_of_appetite,-0.026770,Muscle_ache,-0.002865,Rhinorrhea,-0.080064


In [13]:
def rank(predictor,blood_type):

    df_temp = df[df['Blood_Group'] == blood_type]
    


    X = df_temp.drop(['Blood_Group'],axis=1)


    X = pd.DataFrame(X[predictor])
    #print(X)
    X['Outcome'] = label_Outcome_mapped


    
    corr_val = X['Outcome'].corr(X[predictor])
    
    return corr_val





blood_types = ["A+","A-","AB+","B+","B-","O+","O-"]

dataframes = []

BloodType_df_abs = pd.DataFrame()
for blood_type in blood_types:
    print(blood_type,"-----------------------")
    list_of_AUC = []
    for feature in df.columns:
        if feature == "Blood_Group":
            continue
        #["Age"," Incubation period (days)","Cyanosis","AP","ARF","AT","COPD","DM","DMHT","HT","NONE"]:
        list_of_AUC.append({blood_type+" Feature":feature, blood_type+" Corr" : rank(feature,blood_type)})

        
    df_temp = pd.DataFrame(list_of_AUC)
    df_temp[blood_type+' Corr'] = abs(df_temp[blood_type+' Corr'])
    
    df_temp = df_temp.sort_values(by=[blood_type+' Corr',blood_type+' Feature'], ascending=False)
    
#     df = df.dropna(axis=0, how='any')
    
    df_temp = df_temp.reset_index()
    
    
    
    dataframes.append(df_temp)
    
    
    

    BloodType_df_abs[blood_type+' Feature'] = df_temp[blood_type+' Feature']
    BloodType_df_abs[blood_type+' Corr'] = df_temp[blood_type+' Corr']
    #print(df_temp)



#https://towardsdatascience.com/feature-selection-with-boruta-in-python-676e3877e596



A+ -----------------------
A- -----------------------
AB+ -----------------------
B+ -----------------------
B- -----------------------
O+ -----------------------
O- -----------------------


In [19]:

BloodType_df_abs= BloodType_df_abs[~BloodType_df_abs.isnull().any(axis=1)]

,A+ Feature,A+ Corr,A- Feature,A- Corr,AB+ Feature,AB+ Corr,B+ Feature,B+ Corr,B- Feature,B- Corr,O+ Feature,O+ Corr,O- Feature,O- Corr
0,Age,0.277960,Age,0.425124,Age,0.331944,Age,0.201408,Cyanosis,0.447166,Cyanosis,0.212281,Cyanosis,0.859990
1,Cyanosis,0.191042,Headache,0.394591,Cyanosis,0.280211,Cyanosis,0.175498,Age,0.167309,Age,0.206141,Headache,0.305556
2,Asthenia,0.038260,Cyanosis,0.368593,Rhinorrhea,0.057374,Cough,0.068726,Diarrhea,0.131764,is_Male,0.060103,Age,0.278516
3,Diarrhea,0.036801,Cough,0.191854,Asthenia,0.051399,is_Male,0.038256,Anosmia_Ageusia,0.115584,Chills,0.045931,is_Male,0.157552
4,Headache,0.025983,Fever,0.142987,Anosmia_Ageusia,0.049281,Dyspnea,0.031712,Fever,0.078716,Anosmia_Ageusia,0.043874,Chills,0.148690
5,Sore_throat,0.022475,Diarrhea,0.114198,Dyspnea,0.037259,Headache,0.029602,Headache,0.065705,Diarrhea,0.034310,Diarrhea,0.123091
6,Chills,0.020619,Muscle_ache,0.059803,is_Male,0.034234,Anosmia_Ageusia,0.022365,Nausea_vomiting,0.060365,Sore_throat,0.030116,Sore_throat,0.096511
7,Anosmia_Ageusia,0.015020,Rhinorrhea,0.059803,Cough,0.030780,Muscle_ache,0.021706,Sore_throat,0.059894,Dyspnea,0.027543,Muscle_ache,0.089087
8,Muscle_ache,0.013703,Asthenia,0.058532,Fever,0.028949,Chills,0.021175,Cough,0.056953,Nausea_vomiting,0.027288,Rhinorrhea,0.080064
9,Symptomatic,0.012408,Nausea_vomiting,0.055945,Diarrhea,0.026463,Sore_throat,0.020575,Chills,0.052152,Fever,0.024170,Anosmia_Ageusia,0.060193


In [16]:
df_topFive = BloodType_df_abs.head(5)

In [17]:
feature_names = [x for x in BloodType_df.head(8).columns if 'Feature' in x]
df_topFive = BloodType_df.head(5)
df_topFive = df_topFive[feature_names]
df_topFive

,A+ Feature,A- Feature,AB+ Feature,B+ Feature,B- Feature,O+ Feature,O- Feature
0,Diarrhea,Headache,Rhinorrhea,Cough,Diarrhea,is_Male,Headache
1,Anosmia_Ageusia,Cough,Cough,is_Male,Anosmia_Ageusia,Chills,Chills
2,Dyspnea,Diarrhea,Fever,Muscle_ache,Headache,Anosmia_Ageusia,Sore_throat
3,Asymtomatic,Muscle_ache,Diarrhea,Chills,Chills,Diarrhea,Nausea_vomiting
4,Asymptomatic,Rhinorrhea,Asymptomatic,Asymptomatic,Rhinorrhea,Rhinorrhea,Asthenia


In [18]:
for i in df_topFive.columns:
    print(i," : ",list(df_topFive[i]),",")

A+ Feature  :  ['Diarrhea', 'Anosmia_Ageusia', 'Dyspnea', 'Asymtomatic', 'Asymptomatic'] ,
A- Feature  :  ['Headache', 'Cough', 'Diarrhea', 'Muscle_ache', 'Rhinorrhea'] ,
AB+ Feature  :  ['Rhinorrhea', 'Cough', 'Fever ', 'Diarrhea', 'Asymptomatic'] ,
B+ Feature  :  ['Cough', 'is_Male', 'Muscle_ache', 'Chills', 'Asymptomatic'] ,
B- Feature  :  ['Diarrhea', 'Anosmia_Ageusia', 'Headache', 'Chills', 'Rhinorrhea'] ,
O+ Feature  :  ['is_Male', 'Chills', 'Anosmia_Ageusia', 'Diarrhea', 'Rhinorrhea'] ,
O- Feature  :  ['Headache', 'Chills', 'Sore_throat', 'Nausea_vomiting', 'Asthenia'] ,
